In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('../input/chapter-2-carpricedata/data.csv')
df


In [ ]:
# Makes reading the column headings much easier
df.head().T

In [ ]:
df_hw = df[['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg']]
df_hw.columns

In [ ]:
df_hw.columns = df_hw.columns.str.lower().str.replace(' ', '_')
df_hw.columns

In [ ]:
# Rename MSRP value to price
# 
# Could also be done with pd.rename
# df.rename(columns={'MSRP': 'price'})
# 
# 
df_price = df
df_price['price'] = df['MSRP']
del df_price['MSRP']
df_price

In [ ]:
df_hw.isnull().sum()
# Fill in empty values for engine_hp and engine_cylinders as 0

In [ ]:
df_hw = df_hw.fillna(0)
df_hw.isnull().sum()

In [ ]:
#  What is the most frequent observation (mode) for the column transmission_type?
df_hw.transmission_type.value_counts()
# Automatic

In [ ]:
df_hw.dtypes

In [ ]:
categorical_columns = list(df_hw.dtypes[df_hw.dtypes  == 'object'].index)
categorical_columns

In [ ]:
numerical_columns = list(df_hw.dtypes[(df_hw.dtypes == 'int64') | (df_hw.dtypes == 'float64')].index)
numerical_columns


In [ ]:
from IPython.display import display    
df_hw = df_hw.reset_index(drop=True)
df_hw
if isinstance(df_hw, pd.DataFrame):
  print('df is a DataFrame')
else:
  print('df is not a DataFrame')

In [ ]:
df_hw[numerical_columns].corrwith(df_hw['year'])

In [ ]:
# Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, 
# you compute the correlation coefficient between every pair of features in the dataset.
for col in numerical_columns:
    print(col)
    display(df_hw[numerical_columns].corrwith(df_hw[col]))
    print()
    print()
# highwaympg and city_mpg